# CNN-FaceNet-Face-Recognition

Welcome to a new CNN project!

In this project, we are going to use pre-trained model [FaceNet](https://arxiv.org/pdf/1503.03832.pdf) to solve two common face recognition problems:
- **Face Verification** "Is this the claimed person?" For example, at some airports, you can pass through customs by letting a system scan your passport and then verifying that you (the person carrying the passport) are the correct person. A mobile phone that unlocks using your face is also using face verification. This is a 1:1 matching problem.

- **Face Recognition** "Who is this person?" For example, the video lecture showed a [face recognition video](https://www.youtube.com/watch?v=wr4rx0Spihs) of Baidu employees entering the office without needing to otherwise identify themselves. This is a 1:K matching problem.



## Table of Contents
- [1 - Set up the working directory & Import packages ](#1)
- [2 - Load pre-trained model ](#2)
- [3 - Build the database of the encoded images](#3)
- [4 - Apply the model for Face Verification](#4)
- [5 - Apply the model for Face Regconition](#5)


<a name='1'></a>
## 1 - Set up the working directory & Import packages ##

In [2]:
# Move to the working directory on Google Drive as using Google Colab
import os

if 'google.colab' in str(get_ipython()):
  print('Running on CoLab')
  PROJECT_ROOT ="/content/drive/MyDrive/GitHub/CNN-FaceNet-Face-Recognition"
  os.chdir(PROJECT_ROOT)
  !pwd
else:
  PROJECT_ROOT ="."

Running on CoLab
/content/drive/MyDrive/GitHub/CNN-FaceNet-Face-Recognition


In [3]:
# View Nvidia CUDA drivers
!nvidia-smi

Thu Sep  2 03:39:15 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
from tensorflow.keras.models import model_from_json
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import PIL

%matplotlib inline
%load_ext autoreload
%autoreload 2


<a name='2'></a>
## 2 - Load Pre-trained model to Encode a Face Image into a 128-Dimensional Vector##


In [7]:
# Load the model
json_file = open(PROJECT_ROOT + '/keras-facenet-h5/model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights(PROJECT_ROOT + '/keras-facenet-h5/model.h5')

In [8]:
# Map each person's name to a 128-dimensional encoding of their face.
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0)
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

<a name='3'></a>
## 3 - Build the database of the encoded images ##


In [13]:
# build the database
database = {}
database["danielle"] = img_to_encoding(PROJECT_ROOT + "/images/danielle.png", model)
database["younes"] = img_to_encoding(PROJECT_ROOT + "/images/younes.jpg", model)
database["tian"] = img_to_encoding(PROJECT_ROOT + "/images/tian.jpg", model)
database["andrew"] = img_to_encoding(PROJECT_ROOT + "/images/andrew.jpg", model)
database["kian"] = img_to_encoding(PROJECT_ROOT + "/images/kian.jpg", model)
database["dan"] = img_to_encoding(PROJECT_ROOT + "/images/dan.jpg", model)
database["sebastiano"] = img_to_encoding(PROJECT_ROOT + "/images/sebastiano.jpg", model)
database["bertrand"] = img_to_encoding(PROJECT_ROOT + "/images/bertrand.jpg", model)
database["kevin"] = img_to_encoding(PROJECT_ROOT + "/images/kevin.jpg", model)
database["felix"] = img_to_encoding(PROJECT_ROOT + "/images/felix.jpg", model)
database["benoit"] = img_to_encoding(PROJECT_ROOT + "/images/benoit.jpg", model)
database["arnaud"] = img_to_encoding(PROJECT_ROOT + "/images/arnaud.jpg", model)

<a name='4'></a>
## 4 - Apply the model for Face Verification ##

 Build a function that verifies if the person on the "image_path" image is "identity".

In [22]:
def verify(image_path, identity, database, model,dist_thres=1.0):
    """
    Function that verifies if the person on the "image_path" image is "identity".
    
    Arguments:
        image_path -- path to an image
        identity -- string, name of the person you'd like to verify the identity. Has to be an employee who works in the office.
        database -- python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
        model -- your Inception model instance in Keras
    
    Returns:
        dist -- distance between the image_path and the image of "identity" in the database.
        door_open -- True, if the door should open. False otherwise.
    """
    ### START CODE HERE
    # Step 1: Compute the encoding for the image. Use img_to_encoding() see example above. (≈ 1 line)
    encoding = img_to_encoding(image_path, model)
    # Step 2: Compute distance with identity's image (≈ 1 line)
    dist = np.linalg.norm(encoding-database[identity])
    # Step 3: Open the door if dist < 1.0, else don't open (≈ 3 lines)
    if dist < dist_thres:
        print("Correct! It's " + str(identity))
        door_open = True
    else:
        print("Incorrect! It's not " + str(identity))
        door_open = False
    ### END CODE HERE        
    return dist, door_open

Testing

In [23]:
recorded_person = PROJECT_ROOT + "/images/camera_0.jpg"
verify(recorded_person, "younes", database, model)

Correct! It's younes


(0.599294, True)

<a name='5'></a>
## 5 - Apply the model for Face Regconition##

 Build a function that finds who is the person on the image_path image.

In [24]:
def who_is_it(image_path, database, model,dist_thres=1.0):
    """
    Implements face recognition for the office by finding who is the person on the image_path image.
    
    Arguments:
        image_path -- path to an image
        database -- database containing image encodings along with the name of the person on the image
        model -- your Inception model instance in Keras
    
    Returns:
        min_dist -- the minimum distance between image_path encoding and the encodings from the database
        identity -- string, the name prediction for the person on image_path
    """
    
    ### START CODE HERE

    ## Step 1: Compute the target "encoding" for the image. Use img_to_encoding() see example above. ## (≈ 1 line)
    encoding =  img_to_encoding(image_path, model)
    
    ## Step 2: Find the closest encoding ##
    
    # Initialize "min_dist" to a large value, say 100 (≈1 line)
    min_dist = 100
    
    # Loop over the database dictionary's names and encodings.
    for (name, db_enc) in database.items():
        
        # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
        dist = np.linalg.norm(encoding-db_enc)

        # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
        if dist<min_dist:
            min_dist = dist
            identity = name
    ### END CODE HERE
    
    if min_dist > dist_thres:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))
        
    return min_dist, identity

 Let's see if our who_it_is() algorithm identifies a person in the database.

In [25]:
# Test 1 with Younes pictures 
recorded_person = PROJECT_ROOT + "/images/camera_0.jpg"
who_is_it(recorded_person, database, model)

it's younes, the distance is 0.599294


(0.599294, 'younes')